# CIE Project B - Group 4

# 

## Importing Necessary Libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy 
import scipy.io
from scipy.io import loadmat
from sklearn.preprocessing import MinMaxScaler
import time
import pickle
import re
import glob
import PIL
import os.path
import math
from keras.models import load_model
from PIL import Image
from numpy import mean
from numpy import std

In [2]:
import tensorflow as tf
import sklearn
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.neighbors import KNeighborsRegressor
from tensorflow.keras import Sequential, layers
from tensorflow.keras.layers import Flatten, Dense, Conv2D, MaxPool2D, Input, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold, RepeatedKFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from scipy.ndimage.interpolation import shift

## User Functions

In [3]:
def readmat(path):
    
    """
    Reads .mat files and saves them in a dictionary
    
    """
    d = {}
    label = {}
    
    for root, dirs, files in os.walk(path, topdown = True):
        
        for name in files:
            mat=None
            
            file_path = os.path.join(root, name)
            
            key = file_path.split('\\')[-1][:-4]
            
            if 'EPOT' in key:
                mat = loadmat(file_path)
                coords=[float(s) for s in re.findall(r'\d+', key)]
                d[key] = mat['num_data']
                label[key] = np.array(coords).reshape(1,-1)
                
            if 'impact' in key:
                mat = np.loadtxt(file_path, delimiter= '\t', skiprows = 8)
                mat[:, [3, 4]] = mat [:, [4, 3]]
                exp_name=[s for s in re.findall(r'\d+', key)]
                d[key] =  mat
                label[key] = np.array(exp_name)
        
    return d, label

In [4]:
def augmenting(input_dict,label_dict,x,y):
    """
    Function to augment the data
    """
    
    augmented_input_list = [] # List to save all input data points
    augmented_label_list = [] # List to save all input label points
    centre               = np.array([[x,y]]) # Center of drop area = (250, 250)
    
    temp       = np.array(list(input_dict.values()))
    temp_label = np.array(list(label_dict.values())) - centre
     
    temp_new       = temp
    temp_label_new = temp_label
    
    # Avoid duplication of drop at origin
    origin = np.where(np.all(temp_label_new==0, axis=(1, 2)))[0]
    if (origin.size):
        augmented_input_list.extend(list(temp_new[origin,:,:]))
        augmented_label_list.extend(list(temp_label_new[origin,:,:])) 
        temp_new=np.delete(temp_new,origin,axis=0)
        temp_label_new=np.delete(temp_label_new,origin,axis=0)

    # Avoid duplication of drops on X-axis
    x_axis = np.where(temp_label_new[:,:,1]==0)[0]
    if (x_axis.size):
        changer       = temp_new[x_axis,:,:]
        changer_label = temp_label_new[x_axis,:,:]
        augmented_input_list.extend(list(changer[:, :, [0,1,2,3,4]]))
        augmented_input_list.extend(list(changer[:, :, [0,1,4,3,2]]))
        augmented_label_list.extend(list(changer_label)) 
        augmented_label_list.extend(list(np.stack((-changer_label[:,:,0].reshape((-1,1,1)), 
                                                   changer_label[:,:,1].reshape((-1,1,1))), axis=2).reshape((-1,1,2))))    
        temp_new       = np.delete(temp_new,x_axis,axis=0)
        temp_label_new = np.delete(temp_label_new,x_axis,axis=0)
    
    # Avoid duplication of drops on Y-axis
    y_axis = np.where(temp_label_new[:,:,0]==0)[0]
    if (y_axis.size):
        changer       = temp_new[y_axis,:,:]
        changer_label = temp_label_new[y_axis,:,:]       
        augmented_input_list.extend(list(changer[:, :, [0,1,2,3,4]]))
        augmented_input_list.extend(list(changer[:, :, [0,3,2,1,4]]))
        augmented_label_list.extend(list(changer_label)) 
        augmented_label_list.extend(list(np.stack((changer_label[:,:,0].reshape((-1,1,1)), 
                                                   -changer_label[:,:,1].reshape((-1,1,1))), axis=2).reshape((-1,1,2))))               
        temp_new       = np.delete(temp_new,y_axis,axis=0)
        temp_label_new = np.delete(temp_label_new,y_axis,axis=0)
        
        
    augmented_input_list.extend(list(temp_new[:,:,[0,1,2,3,4]])) # 1st Quadrant
    augmented_input_list.extend(list(temp_new[:,:,[0,1,4,3,2]])) # 2nd Quadrant   
    augmented_input_list.extend(list(temp_new[:,:,[0,3,4,1,2]])) # 3rd Quadrant
    augmented_input_list.extend(list(temp_new[:,:,[0,3,2,1,4]])) # 4th Quadrant
    
    augmented_label_list.extend(list(temp_label_new))   
    augmented_label_list.extend(list(np.stack((-temp_label_new[:,:,0].reshape((-1,1,1)),
                                               temp_label_new[:,:,1].reshape((-1,1,1))), axis=2).reshape((-1,1,2))))   
    augmented_label_list.extend(list(-temp_label_new)) 
    augmented_label_list.extend(list(np.stack((temp_label_new[:,:,0].reshape((-1,1,1)),
                                               -temp_label_new[:,:,1].reshape((-1,1,1))), axis=2).reshape((-1,1,2))))    

       
    return augmented_input_list, augmented_label_list

In [5]:
def expt_cutter(expt_data, expt_label):
    
    """
    Cuts the Experimental Data for a time window of 0.2ms
    The EPOT Data is also available for a time window of 0.2ms, hence a window of 0.2ms is chosen for Experimental Data
    """

    expt_cut_data  = []
    expt_cut_label = []
    
    for key in expt_data:
        
        expt_sliced = expt_data[key][49980:50180, :]
        
        expt_cut_data.append(expt_sliced)  
        expt_cut_label.append(expt_label[key])
    
    return expt_cut_data, expt_cut_label

In [6]:
def resample_numerical(inp_data, inp_label, num_points):
    
    """
    Resamples the given data to specified number of points.
    Can upsample (usually for experimental data) as well as downsample (usually for numerical EPOT data)
    """
    resampled_data  = []
    
    for i in range(len(inp_data)):
        
        f1 = scipy.interpolate.interp1d(inp_data[i][:,0], inp_data[i][:,1], bounds_error = False, kind = 'quadratic')
        f2 = scipy.interpolate.interp1d(inp_data[i][:,0], inp_data[i][:,2], bounds_error = False, kind = 'quadratic')
        f3 = scipy.interpolate.interp1d(inp_data[i][:,0], inp_data[i][:,3], bounds_error = False, kind = 'quadratic')
        f4 = scipy.interpolate.interp1d(inp_data[i][:,0], inp_data[i][:,4], bounds_error = False, kind = 'quadratic')
        
        t_new = np.linspace(inp_data[i][0,0], inp_data[i][-1,0], num_points)
        
        c1 = f1(t_new)
        c2 = f2(t_new)
        c3 = f3(t_new)
        c4 = f4(t_new)
        
        t_new = np.reshape(t_new, (t_new.shape[0], 1))
        c1 = np.reshape(c1, (c1.shape[0], 1))
        c2 = np.reshape(c2, (c2.shape[0], 1))
        c3 = np.reshape(c3, (c3.shape[0], 1))
        c4 = np.reshape(c4, (c4.shape[0], 1))
        
        arr = np.hstack((t_new, c1, c2, c3, c4))
        
        resampled_data.append(arr)  
        
    resampled_label = inp_label
    
    return resampled_data, resampled_label

In [7]:
def resample_expt(inp_data, inp_label, num_points):
    
    """
    Resamples the given data to specified number of points.
    Can upsample (usually for experimental data) as well as downsample (usually for numerical EPOT data)
    """
    resampled_data  = []
    resampled_label = []
    
    for i in range(len(inp_data)):
        
        f1 = scipy.interpolate.interp1d(inp_data[i][:,0], inp_data[i][:,1], bounds_error = False, kind = 'quadratic')
        f2 = scipy.interpolate.interp1d(inp_data[i][:,0], inp_data[i][:,2], bounds_error = False, kind = 'quadratic')
        f3 = scipy.interpolate.interp1d(inp_data[i][:,0], inp_data[i][:,3], bounds_error = False, kind = 'quadratic')
        f4 = scipy.interpolate.interp1d(inp_data[i][:,0], inp_data[i][:,4], bounds_error = False, kind = 'quadratic')
        
        t_new = np.linspace(inp_data[i][0,0], inp_data[i][-1,0], num_points)
        
        c1 = f1(t_new)
        c2 = f2(t_new)
        c3 = f3(t_new)
        c4 = f4(t_new)
        
        t_new = np.reshape(t_new, (t_new.shape[0], 1))
        c1 = np.reshape(c1, (c1.shape[0], 1))
        c2 = np.reshape(c2, (c2.shape[0], 1))
        c3 = np.reshape(c3, (c3.shape[0], 1))
        c4 = np.reshape(c4, (c4.shape[0], 1))
        
        arr = np.hstack((t_new, c1, c2, c3, c4))
        
        label = np.array(inp_label[i][:-1], dtype='float') - np.array([250, 250])
        label = np.reshape(label, (1,2))
        
        resampled_data.append(arr)  
        resampled_label.append(label)

    return resampled_data, resampled_label

In [8]:
def resample_expt_only(inp_data, inp_label, num_points):
    
    """
    Resamples the given data to specified number of points.
    Can upsample (usually for experimental data) as well as downsample (usually for numerical EPOT data)
    """
    resampled_data  = []
    resampled_label = []
    
    for i in range(len(inp_data)):
        
        f1 = scipy.interpolate.interp1d(inp_data[i][:,0], inp_data[i][:,1], bounds_error = False, kind = 'quadratic')
        f2 = scipy.interpolate.interp1d(inp_data[i][:,0], inp_data[i][:,2], bounds_error = False, kind = 'quadratic')
        f3 = scipy.interpolate.interp1d(inp_data[i][:,0], inp_data[i][:,3], bounds_error = False, kind = 'quadratic')
        f4 = scipy.interpolate.interp1d(inp_data[i][:,0], inp_data[i][:,4], bounds_error = False, kind = 'quadratic')
        
        t_new = np.linspace(inp_data[i][0,0], inp_data[i][-1,0], num_points)
        
        c1 = f1(t_new)
        c2 = f2(t_new)
        c3 = f3(t_new)
        c4 = f4(t_new)
        
        t_new = np.reshape(t_new, (t_new.shape[0], 1))
        c1 = np.reshape(c1, (c1.shape[0], 1))
        c2 = np.reshape(c2, (c2.shape[0], 1))
        c3 = np.reshape(c3, (c3.shape[0], 1))
        c4 = np.reshape(c4, (c4.shape[0], 1))
        
        arr = np.hstack((t_new, c1, c2, c3, c4))
        
        label = np.array(inp_label[i][:], dtype='float')
        
        resampled_data.append(arr)  
        resampled_label.append(label)

    return resampled_data, resampled_label

In [9]:
def normalizing_datamap(input_list):    

    """
    Normalize the input lists based on whole matrix max and min
    """
    
    scaler_datamap = MinMaxScaler((-1,1))

    normalized_input_datamap = np.empty_like(np.array(input_list))
    count = 0
 
    for i in input_list:

        tempnow = i[:,1:]
        data    = 2 * (tempnow-np.min(tempnow)) / (np.max(tempnow)-np.min(tempnow)) - 1
        
        normalized_input_datamap[count,:,0] = i[:,0]
        normalized_input_datamap[count,:,1:] = data
        
        count += 1
        
    transformer = (normalized_input_datamap.shape[0],normalized_input_datamap.shape[1],normalized_input_datamap.shape[2],1)
    normalized_input_datamap = normalized_input_datamap.reshape(transformer)

#     print("\nNormalizing done for the data")

    return normalized_input_datamap

In [10]:
def normalizing_datamap_01(input_list):    

    """
    Normalize the input lists based on whole matrix max and min
    """
    
    scaler_datamap = MinMaxScaler((-1,1))

    normalized_input_datamap = np.empty_like(np.array(input_list))
    count = 0
 
    for i in input_list:

        tempnow = i[:,1:]
        data    =  (tempnow-np.min(tempnow)) / (np.max(tempnow)-np.min(tempnow))
        
        normalized_input_datamap[count,:,0] = i[:,0]
        normalized_input_datamap[count,:,1:] = data
        
        count += 1
        
    transformer = (normalized_input_datamap.shape[0],normalized_input_datamap.shape[1],normalized_input_datamap.shape[2],1)
    normalized_input_datamap = normalized_input_datamap.reshape(transformer)

#     print("\nNormalizing done for the data")

    return normalized_input_datamap

In [11]:
def normalizing_labels(res_label):
    
    """
    Normalizes the labels
    """
    out_mm = np.array(res_label) / 75.0
    out_mm = np.squeeze(out_mm, axis=1)
    return out_mm

In [12]:
def cut_num_data(list_input,threshold = 0.0005):
    input_num_data=[]
#     hurra=np.empty((925,1))
#     hurra2=np.empty((925,1))

#     kaai=[]
#     kaai.append(list_input[0])
    count=0
    for i in list_input:
        temp=i[:,1:]
#         print(temp)
        temp_diff=np.abs(np.diff(temp, axis=0))
#         print(temp_diff)
        temp_tocheck=np.max(temp_diff,axis=1)
#         print(temp_tocheck.shape)
        arg=np.argmax(temp_tocheck>=threshold)
#         print(arg)
        arg_end=np.argmax(temp_tocheck[::-1]>=threshold)
#         print(arg_end)

        input_num_data.append(i[(1+arg):(i.shape[0]-arg_end+1),:])
#         hurra[count,0]=np.max(temp_diff)
#         hurra2[count,0]=np.min(temp_diff)

        temp = temp_diff = temp_tocheck = arg = None
#         arg_end=None
        count+=1
    
#     print((np.max(hurra)))
#     print((np.min(hurra2)))
    
    return input_num_data


In [13]:
def cut_expt_data(list_input,threshold = 0.05):
    input_num_data=[]
#     hurra=np.empty((925,1))
#     hurra2=np.empty((925,1))

#     kaai=[]
#     kaai.append(list_input[0])
    count=0
    for i in list_input:
        temp=i[:,1:]
#         print(temp)
        temp_diff=np.abs(np.diff(temp, axis=0))
#         print(temp_diff)
        temp_tocheck=np.max(temp_diff,axis=1)
#         print(temp_tocheck.shape)
        arg=np.argmax(temp_tocheck>=threshold)
#         print(arg)
#         arg_end=np.argmax(temp_tocheck[::-1]>=threshold)
#         print(arg_end)

        input_num_data.append(i[(1+arg):(1+arg+190),:])
#         hurra[count,0]=np.max(temp_diff)
#         hurra2[count,0]=np.min(temp_diff)

        temp = temp_diff = temp_tocheck = arg = None
#         arg_end=None
        count+=1
    
#     print((np.max(hurra)))
#     print((np.min(hurra2)))
    
    return input_num_data


In [14]:
def augmenting_expt_val(input_expt_list,label_expt_list):
    """
    Function to augment the experimental validation data
    """
    
    augmented_input_expt_list = [] # List to save all input data points
    augmented_label_expt_list = [] # List to save all input label points
    
    
    
    for i,j in zip(input_expt_list,label_expt_list):
        j=j.reshape((1,2))
        if j[0,0]==0 and j[0,1]==0:
            augmented_input_expt_list.append(i)
            augmented_label_expt_list.append(j)
        elif j[0,0]>=0 and j[0,1] >0:
            if j[0,0]==0:
                augmented_label_expt_list.append(j)
                augmented_label_expt_list.append(np.array([[0,-j[0,1]]]))
                
                augmented_input_expt_list.append(i)
                augmented_input_expt_list.append(i[:,[2,1,0,3]])
            else:
                augmented_label_expt_list.append(j)
                augmented_label_expt_list.append(np.array([[-j[0,0],j[0,1]]]))
                augmented_label_expt_list.append(-j)
                augmented_label_expt_list.append(np.array([[j[0,0],-j[0,1]]]))
                
                
                augmented_input_expt_list.append(i)
                augmented_input_expt_list.append(i[:,[0,3,2,1]])
                augmented_input_expt_list.append(i[:,[2,3,0,1]])
                augmented_input_expt_list.append(i[:,[2,1,0,3]])
        elif j[0,0]<0 and j[0,1] >= 0:
            if j[0,1]==0:
                augmented_label_expt_list.append(j)
                augmented_label_expt_list.append(np.array([[-j[0,0],0]]))
                
                augmented_input_expt_list.append(i)
                augmented_input_expt_list.append(i[:,[0,3,2,1]])
            else:
                augmented_label_expt_list.append(j)
                augmented_label_expt_list.append(np.array([[j[0,0],-j[0,1]]]))
                augmented_label_expt_list.append(-j)
                augmented_label_expt_list.append(np.array([[-j[0,0],j[0,1]]]))
                
                
                augmented_input_expt_list.append(i)
                augmented_input_expt_list.append(i[:,[2,1,0,3]])
                augmented_input_expt_list.append(i[:,[2,3,0,1]])
                augmented_input_expt_list.append(i[:,[0,3,2,1]])
                
        elif j[0,0]<=0 and j[0,1] < 0:
            if j[0,0]==0:
                augmented_label_expt_list.append(j)
                augmented_label_expt_list.append(np.array([[0,-j[0,1]]]))
                
                augmented_input_expt_list.append(i)
                augmented_input_expt_list.append(i[:,[2,1,0,3]])
            else:
                augmented_label_expt_list.append(j)
                augmented_label_expt_list.append(np.array([[-j[0,0],j[0,1]]]))
                augmented_label_expt_list.append(-j)
                augmented_label_expt_list.append(np.array([[j[0,0],-j[0,1]]]))
                
                
                augmented_input_expt_list.append(i)
                augmented_input_expt_list.append(i[:,[0,3,2,1]])
                augmented_input_expt_list.append(i[:,[2,3,0,1]])
                augmented_input_expt_list.append(i[:,[2,1,0,3]])
                
                
        elif j[0,0]>0 and j[0,1] <= 0:
            if j[0,1]==0:
                augmented_label_expt_list.append(j)
                augmented_label_expt_list.append(np.array([[-j[0,0],0]]))
                
                augmented_input_expt_list.append(i)
                augmented_input_expt_list.append(i[:,[0,3,2,1]])
            else:
                augmented_label_expt_list.append(j)
                augmented_label_expt_list.append(np.array([[j[0,0],-j[0,1]]]))
                augmented_label_expt_list.append(-j)
                augmented_label_expt_list.append(np.array([[-j[0,0],j[0,1]]]))
                
                
                augmented_input_expt_list.append(i)
                augmented_input_expt_list.append(i[:,[2,1,0,3]])
                augmented_input_expt_list.append(i[:,[2,3,0,1]])
                augmented_input_expt_list.append(i[:,[0,3,2,1]])

    
    return augmented_input_expt_list, augmented_label_expt_list

In [15]:
def artificial_augmenting(augmented_input_list, augmented_label_list, threshold = 100):
    """
    Artificial Augmenting: Adding Controlled Noise to the datapoint
    
    shifted_add_s1: Shift Sensor 1 graph to the right by 'threshold' points --> 4 extra datapoints
    shifted_sub_s1: Shift Sensor 1 graph to the left by 'threshold' points  --> 4 extra datapoints
    shifted_add_s12: Shift Sensor 1 & 2 to the right by 'threshold' points  --> 6 extra datapoints
    shifted_sub_s12: Shift Sensor 1 & 2 to the left by 'threshold' points   --> 6 extra datapoints
                                                                            ------------------------
                                                                     Total  --> 20 extra data points per drop data
    Before artificial augmenting = 925
    After artificial augmenting  = 925 + 20*925 = 19425
    
    Final augmented list Format:
    i = 0:925 --> Original Data
    i = 925 + 20*d + c : c=00-->S1 to right     c=01-->S2 to right     c=02-->S3 to right     c=03-->S4 to right
                         c=04-->S1 to left      c=05-->S2 to left      c=06-->S3 to left      c=07-->S4 to left                    
                         c=08-->S1,S2 to right  c=09-->S1,S3 to right  c=10-->S1,S4 to right 
                                                c=11-->S2,S3 to right  c=12-->S2,S4 to right
                                                                       c=13-->S3,S4 to right
                         c=14-->S1,S2 to left   c=15-->S1,S3 to left   c=16-->S1,S4 to left 
                                                c=17-->S2,S3 to left   c=18-->S2,S4 to left
                                                                       c=19-->S3,S4 to left
    """
    
    artificial_augmented_input_list = augmented_input_list.copy()
    artificial_augmented_label_list = augmented_label_list.copy() 
#     cnt = 0
    for i, j in zip(augmented_input_list, augmented_label_list):
        
        # Making copies of the original datamap
        shifted_add_s1 = i.copy()
        shifted_add_s2 = i.copy()
        shifted_add_s3 = i.copy()
        shifted_add_s4 = i.copy()
        
#         shifted_sub_s1 = i.copy()
#         shifted_sub_s2 = i.copy()
#         shifted_sub_s3 = i.copy()
#         shifted_sub_s4 = i.copy()
        
        # Roll the sensor axes one-by-one
        shifted_add_sensor_s1 = shift(i[:,1], threshold, cval=0.0)
        shifted_add_sensor_s2 = shift(i[:,2], threshold, cval=0.0)
        shifted_add_sensor_s3 = shift(i[:,3], threshold, cval=0.0)
        shifted_add_sensor_s4 = shift(i[:,4], threshold, cval=0.0)
        
#         shifted_sub_sensor_s1 = shift(i[:,1], -threshold, cval=0.0)
#         shifted_sub_sensor_s2 = shift(i[:,2], -threshold, cval=0.0)
#         shifted_sub_sensor_s3 = shift(i[:,3], -threshold, cval=0.0)
#         shifted_sub_sensor_s4 = shift(i[:,4], -threshold, cval=0.0)
        
        # Shift 1 sensor to right
        shifted_add_s1[:,1] = shifted_add_sensor_s1
        shifted_add_s2[:,2] = shifted_add_sensor_s2
        shifted_add_s3[:,3] = shifted_add_sensor_s3
        shifted_add_s4[:,4] = shifted_add_sensor_s4
        
        # Shift 1 sensor to left
#         shifted_sub_s1[:,1] = shifted_sub_sensor_s1
#         shifted_sub_s2[:,2] = shifted_sub_sensor_s2
#         shifted_sub_s3[:,3] = shifted_sub_sensor_s3
#         shifted_sub_s4[:,4] = shifted_sub_sensor_s4

        # Shift 2 sensors to right
        shifted_add_s12      = shifted_add_s1.copy()
        shifted_add_s12[:,2] = shifted_add_sensor_s2
        
        shifted_add_s13      = shifted_add_s1.copy()
        shifted_add_s13[:,3] = shifted_add_sensor_s3
        
        shifted_add_s14      = shifted_add_s1.copy()
        shifted_add_s14[:,4] = shifted_add_sensor_s4
        
        shifted_add_s23      = shifted_add_s2.copy()
        shifted_add_s23[:,3] = shifted_add_sensor_s3
        
        shifted_add_s24      = shifted_add_s2.copy()
        shifted_add_s24[:,4] = shifted_add_sensor_s4
        
        shifted_add_s34      = shifted_add_s3.copy()
        shifted_add_s34[:,4] = shifted_add_sensor_s4
        
        # Shift 2 sensors to left
#         shifted_sub_s12      = shifted_sub_s1.copy()
#         shifted_sub_s12[:,2] = shifted_sub_sensor_s2
        
#         shifted_sub_s13      = shifted_sub_s1.copy()
#         shifted_sub_s13[:,3] = shifted_sub_sensor_s3
        
#         shifted_sub_s14      = shifted_sub_s1.copy()
#         shifted_sub_s14[:,4] = shifted_sub_sensor_s4
        
#         shifted_sub_s23      = shifted_sub_s2.copy()
#         shifted_sub_s23[:,3] = shifted_sub_sensor_s3
        
#         shifted_sub_s24      = shifted_sub_s2.copy()
#         shifted_sub_s24[:,4] = shifted_sub_sensor_s4
        
#         shifted_sub_s34      = shifted_sub_s3.copy()
#         shifted_sub_s34[:,4] = shifted_sub_sensor_s4
        
        # Append to the list of datapoints
        
#         artificial_augmented_input_list.extend((shifted_add_s1,
#                                                 shifted_add_s2,
#                                                 shifted_add_s3,
#                                                 shifted_add_s4,
#                                                 shifted_sub_s1,
#                                                 shifted_sub_s2,
#                                                 shifted_sub_s3,
#                                                 shifted_sub_s4,
#                                                 shifted_add_s12,
#                                                 shifted_add_s13,
#                                                 shifted_add_s14,
#                                                 shifted_add_s23,
#                                                 shifted_add_s24,
#                                                 shifted_add_s34,
#                                                 shifted_sub_s12,
#                                                 shifted_sub_s13,
#                                                 shifted_sub_s14,
#                                                 shifted_sub_s23,
#                                                 shifted_sub_s24,
#                                                 shifted_sub_s34))
        
        artificial_augmented_input_list.extend((shifted_add_s1,
                                                shifted_add_s2,
                                                shifted_add_s3,
                                                shifted_add_s4,
                                                shifted_add_s12,
                                                shifted_add_s13,
                                                shifted_add_s14,
                                                shifted_add_s23,
                                                shifted_add_s24,
                                                shifted_add_s34))
        
        artificial_augmented_label_list.extend((j,j,j,j,j,j,j,j,j,j))
#         cnt = cnt + 1 
#         print(cnt)
        
    return artificial_augmented_input_list, artificial_augmented_label_list

## Plotting Functions

In [16]:
def plot_saver(dictionary, dots=50):
    
    """
    Plotter for Initial EPOT data
    Plots all plots in one graph
    """
    
    for key, value in dictionary.items():
        
        fig=plt.figure(figsize = (50,25));
        plt.plot(value[:,0], value[:,1], label = 'Sensor P1')
        plt.plot(value[:,0], value[:,2], label = 'Sensor P2')
        plt.plot(value[:,0], value[:,3], label = 'Sensor P3')
        plt.plot(value[:,0], value[:,4], label = 'Sensor P4')
        plt.xticks(fontsize = 20)
        plt.yticks(fontsize = 20)
        plt.xlabel('Time', fontsize = 30)
        plt.ylabel('Sensor Readings (Volts)', fontsize = 30)
        plt.title('Time vs Sensor Data', fontsize = 40)
        plt.grid();
        plt.legend(fontsize = 30);
        if 'EPOT' in key:
            name = r'.\Plots\Num' + '\\' + key + '.jpg'
            plt.savefig(name, dpi = dots)
            plt.close(fig)
        if 'impact' in key:
            name = r'.\Plots\Expt_Val' + '\\' + key + '.jpg'
            plt.savefig(name, dpi = dots)
            plt.close(fig)

In [17]:
def plot_saver_individual(Dictionary, dots=50):
    
    """
    Plotter for Initial EPOT data
    Plots subplots
    """
    
    for key, value in Dictionary.items():
        
        fig,(ax1, ax2, ax3, ax4) = plt.subplots(4,figsize=(30,15))
        ax1.set_title('Sensor 1',loc='right')
        ax2.set_title('Sensor 2',loc='right')
        ax3.set_title('Sensor 3',loc='right')
        ax4.set_title('Sensor 4',loc='right') 
        plt.suptitle("Time vs Sensor Data")
        
        ax1.grid()
        ax2.grid()
        ax3.grid()
        ax4.grid()
        
        ax1.plot(value[:,0],value[:,1])
        ax2.plot(value[:,0],value[:,2])
        ax3.plot(value[:,0],value[:,3])
        ax4.plot(value[:,0],value[:,4])
        
        if 'EPOT' in key:
            name = r'.\Plots\Indi_Num' + '\\' + key + '.jpg'
            plt.savefig(name, dpi = dots)
            plt.close(fig)
        if 'impact' in key:
            name = r'.\Plots\Indi_Expt' + '\\' + key + '.jpg'
            plt.savefig(name, dpi = dots)
            plt.close(fig)

In [18]:
def plot_saver_list_2(augmented_input_list,augmented_label_list, folder, dots = 50):
    
    """
    Plotting function for Augmented Data
    """

    for i, j in zip(augmented_input_list, augmented_label_list):
        
        fig = plt.figure(figsize = (30,15))
        plt.plot(i[:,0], label = 'Sensor P1')
        plt.plot(i[:,1], label = 'Sensor P2')
        plt.plot(i[:,2], label = 'Sensor P3')
        plt.plot(i[:,3], label = 'Sensor P4')

        plt.xticks(fontsize = 20)
        plt.yticks(fontsize = 20)

        plt.xlabel('Time', fontsize = 30)
        plt.ylabel('Sensor Readings (Volts)', fontsize = 30)
        plt.title(str(j), fontsize = 40)

        plt.grid()
        plt.legend(fontsize = 30)

        name = r'.\Plots\\' + folder + '\\' + str(j) + '.jpg'
        plt.savefig(name, dpi = dots)
        
        plt.close(fig)

In [19]:
def plot_saver_list(augmented_input_list,augmented_label_list, folder, dots = 50):
    
    """
    Plotting function for Augmented Data
    """
    counter=1
    
    for i, j in zip(augmented_input_list, augmented_label_list):
        
        fig = plt.figure(figsize = (30,15))
        plt.plot(i[:,0], i[:,1], label = 'Sensor P1')
        plt.plot(i[:,0], i[:,2], label = 'Sensor P2')
        plt.plot(i[:,0], i[:,3], label = 'Sensor P3')
        plt.plot(i[:,0], i[:,4], label = 'Sensor P4')

        plt.xticks(fontsize = 20)
        plt.yticks(fontsize = 20)

        plt.xlabel('Time', fontsize = 30)
        plt.ylabel('Sensor Readings (Volts)', fontsize = 30)
        plt.title(str(j), fontsize = 40)

        plt.grid()
        plt.legend(fontsize = 30)
        

        
        name = r'.\Plots\\' + folder + '\\' + str(counter) + '_' + str(j) + '.jpg'
        
        plt.savefig(name, dpi = dots)
        
        plt.close(fig)
        counter+=1

In [20]:
def plot_saver_list_after(augmented_input_list,augmented_label_list, folder, dots = 50):
    
    """
    Plotting function for Augmented Data
    """
    counter=1
    
    for i, j in zip(augmented_input_list, augmented_label_list):
        
        fig = plt.figure(figsize = (30,15))
        plt.plot(i[:,0],  label = 'Sensor P1')
        plt.plot(i[:,1],  label = 'Sensor P2')
        plt.plot(i[:,2],  label = 'Sensor P3')
        plt.plot(i[:,3],  label = 'Sensor P4')

        plt.xticks(fontsize = 20)
        plt.yticks(fontsize = 20)

        plt.xlabel('Time', fontsize = 30)
        plt.ylabel('Sensor Readings (Volts)', fontsize = 30)
        plt.title(str(j), fontsize = 40)

        plt.grid()
        plt.legend(fontsize = 30)
        

        
        name = r'.\Plots\\' + folder + '\\' + str(counter) + '_' + str(j) + '.jpg'
        
        plt.savefig(name, dpi = dots)
        
        plt.close(fig)
        counter+=1

In [21]:
def plot_scatter(label_list,dots=50):
    
    """
    Scatter Plot representing all drop locations
    """

    fig=plt.figure(figsize = (20,20))
    
    for plott in label_list:
        if np.any(plott==0):
            colour='cyan'

        elif np.all(plott>0):
            colour='red'

        elif np.all(plott<0):
            colour='black'

        elif np.any(plott<0):
            if plott[0,0]<0:
                colour='blue'
            else:
                colour='green'
                
        plt.scatter(plott[0,0],plott[0,1], c=colour ,marker='o')

    plt.axhline(y=0,c='brown')
    plt.axvline(x=0,c='brown')

    plt.xticks(fontsize = 20)
    plt.yticks(fontsize = 20)
    
    plt.title('Augmented data',size = 40)
    plt.grid()
    
    name = r'.\Plots\Scatter' + '\\.jpg'
    plt.savefig(name, dpi = dots)
    plt.close(fig)

In [22]:
def color_map_generator_matlab(input_list,label_list):
   
    """
    Saves Matlab EPOT Data for image generation
    """
    input_export=np.array(input_list)
    label_export=np.array(label_list).reshape((-1,2))

    scipy.io.savemat('Matlab_Input.mat', dict(Input=input_export,label=label_export))

In [23]:
def resizer(path1, path2, height, width):

    """
    Resizes an image to specified height and width
    """

    f  = path1
    f2 = path2

    for file in os.listdir(f):
        f_img  = f+"/"+file
        f2_img = f2+"/"+file
        
        img = Image.open(f_img)
        img = img.resize((width,height))
        img.save(f2_img)

In [24]:
def resampled_plot(resampled_data, augmented_input_list, augmented_label_list):
    
    """
    Function to Plot Resampled and Original EPOT Data on the same graph
    """
    centre=[250.0, 250.0]
    
    for i, j, k in zip(resampled_data, augmented_label_list, augmented_input_list):
        
        color1 = ['black','brown','blue','red']
        color2 = ['yellow','pink','cyan','lawngreen']

        fig = plt.figure(figsize = (40,15));
        for p in range(1,len(i[0])):
            plt.plot(k[:,0], k[:,p], color=color2[p-1],linewidth=6, label='Sesnor P'+str(p)+'Actual')
            plt.plot(i[:,0], i[:,p], color=color1[p-1],linewidth=1, label = 'Sensor P'+str(p)+' resampled')

        plt.xticks(fontsize = 20)
        plt.yticks(fontsize = 20)

        plt.xlabel('Time', fontsize = 30)
        plt.ylabel('Sensor Readings (Volts)', fontsize = 30)

        position = []
        position = j[0]+centre
        plt.title('Coordinate '+str(int(position[0]))+','+str(int(position[1])), fontsize = 40)
        plt.grid()
        plt.legend(fontsize = 30, loc='upper right')
        
        name = r'.\Plots\Resample2' + '\\' + str(int(position[0]))+','+str(int(position[1])) + '.jpg'
        plt.savefig(name, dpi=50)

        plt.close(fig)

In [25]:
def expt_cut_plot(expt_cutdata,cutdata_label):
    """
    Function to plot the cut experimental data
    """
    for key in np.arange(np.array(expt_cutdata).shape[0]):
        fig=plt.figure(figsize=(25,10))
        plt.plot(expt_cutdata[key][:,0],expt_cutdata[key][:,1],label='sensor1')
        plt.plot(expt_cutdata[key][:,0],expt_cutdata[key][:,2],label='sensor2')
        plt.plot(expt_cutdata[key][:,0],expt_cutdata[key][:,3],label='sensor3')
        plt.plot(expt_cutdata[key][:,0],expt_cutdata[key][:,4],label='sensor4')
        plt.title(cutdata_label[key], size=40)
        plt.grid()
        plt.legend()
        name = r'.\Plots\Expt_Val_Cut' + '\\' + cutdata_label[key] + '.jpg'
        plt.savefig(name, dpi = 100)
        plt.close(fig)

## Data Pre-Processing

In [26]:
# Specifying the impact locations
A_impact_x, A_impact_y = 60,60
B_impact_x, B_impact_y = 150,150
Plate_x, Plate_y       = 500,500
samples                = 500 #Number of Resampling Points

In [28]:
# output_expt_test_given=np.array([[245,255],
#                                  [235,235],
#                                  [250,250],
#                                  [255,265],
#                                  [250,265],
#                                  [265,235],
#                                  [300,285],
#                                  [250,250],
#                                  [225,200],
#                                  [260,320],
#                                  [235,235],
#                                  [250,265],
#                                  [265,235],
#                                  [200,225],
#                                  [245,255],
#                                  [270,300],
#                                  [265,255],
#                                  [235,300]])

In [29]:
#Run it only once, to generate the pickle file

# input_data, input_label                             = readmat(r'.\Project_B_Data\EPOT_Data')
# expt_data, expt_label                               = readmat(r'.\\Project_B_Data\Group4')
# augmented_validated_data, augmented_validated_label = readmat(r'.\Project_B_Data\Validation_augmented_data')
# val_expt_data, val_expt_label                       = readmat(r'.\Project_B_Data\Experimental_validation')
# augmented_input_list, augmented_label_list          = augmenting(input_data, input_label, Plate_x*0.5, Plate_y*0.5)

# with open('VariablesAll', 'wb') as f:
#     pickle.dump([input_data, input_label, 
#                  expt_data, expt_label, 
#                  augmented_validated_data, augmented_validated_label, 
#                  val_expt_data, val_expt_label,
# #                  augmented_input_list, augmented_label_list], f)

# with open('VariablesAll_Given', 'wb') as fg:
#     pickle.dump([output_expt_test_given], fg)

In [35]:
with open('VariablesAll', 'rb') as f:
    input_data, input_label, expt_data, expt_label, augmented_validated_data, augmented_validated_label, val_expt_data, val_expt_label, augmented_input_list, augmented_label_list = pickle.load(f)

In [36]:
with open('VariablesAll_Given', 'rb') as fg:
    output_expt_test_given = np.squeeze(np.array(pickle.load(fg)))

In [37]:
# Add this line immediately after loading the 'VariablesAll' pickle file

artificial_augmented_input_list, artificial_augmented_label_list = artificial_augmenting(augmented_input_list, 
                                                                                         augmented_label_list,350)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [38]:
with open('VariablesAll_Given_Augmented_oneside_350', 'wb') as fga:
    pickle.dump([artificial_augmented_input_list, artificial_augmented_label_list], fga)

In [39]:
# with open('VariablesAll_Given_Augmented_oneside', 'rb') as fga:
#     artificial_augmented_input_list, artificial_augmented_label_list = pickle.load(fga)

In [40]:
# Just to check 
print(len(augmented_input_list))
print(len(artificial_augmented_input_list))

925
10175


In [43]:
output_expt_test_given_X = output_expt_test_given[:,0].reshape((output_expt_test_given.shape[0],1))
output_expt_test_given_Y = output_expt_test_given[:,1].reshape((output_expt_test_given.shape[0],1))

In [44]:
val_expt_data = list(val_expt_data.values())
val_expt_label = list(val_expt_label.values())

In [45]:
# cut_data = cut_num_data(augmented_input_list, threshold=0.0008) 

# cut_data = cut_num_data(_augmented_input_list, threshold=0.0008) 
# resampled_data, resampled_label = resample_numerical(cut_data, augmented_label_list, samples)


cut_data = cut_num_data(artificial_augmented_input_list, threshold=0.0008) 
resampled_data, resampled_label = resample_numerical(cut_data, artificial_augmented_label_list, samples)

In [46]:
cut_expt_data_list = cut_expt_data(val_expt_data, threshold=0.05) 
resampled_expt_data, resampled_expt_label = resample_expt(cut_expt_data_list, val_expt_label, samples)

In [47]:
## RUN ONLY ONCE! (Otherwise Restart Kernel) ##

# Cutting and Resampling of Experimental Test Data
expt_data  = list(expt_data.values())
expt_label = list(expt_label.values())

expt_test_cut_data = cut_expt_data(expt_data, threshold= 0.05)
resampled_expt_test_data, resampled_expt_test_label = resample_expt_only(expt_test_cut_data, expt_label, samples)
# plot_saver_list(resampled_data, resampled_label, 'RESAMPLED DATA', dots = 30)

In [48]:
# Normalizing Input Data and Labels
normalized_inp        = normalizing_datamap(resampled_data) 
normalized_out        = normalizing_labels(resampled_label) 

In [49]:
# Normalizing Experimental Data and Labels
normalized_expt_inp        = normalizing_datamap(resampled_expt_data) 
normalized_expt_out        = normalizing_labels(resampled_expt_label)

In [50]:
# Normalizing Test Experimental Data
normalized_expt_test_inp        = normalizing_datamap(resampled_expt_test_data) 

In [51]:
# #Normalising 0-1
# # Normalizing Input Data and Labels
# normalized_inp        = normalizing_datamap_01(resampled_data) 
# normalized_out        = normalizing_labels(resampled_label) 

# # Normalizing Experimental Data and Labels
# normalized_expt_inp        = normalizing_datamap_01(resampled_expt_data) 
# normalized_expt_out        = normalizing_labels(resampled_expt_label)

# # Normalizing Test Experimental Data
# normalized_expt_test_inp        = normalizing_datamap_01(resampled_expt_test_data) 

In [52]:
# Assigning the input and label dataset for CNN
input_normalized  = normalized_inp[:, :, 1:, :]
output_normalized = normalized_out

In [53]:
# Assigning the input and label dataset for CNN
input_expt_normalized  = normalized_expt_inp[:, :, 1:, :]
output_expt_normalized = normalized_expt_out

In [54]:
# Assigning the input and label dataset for CNN
input_expt_test_normalized  = normalized_expt_test_inp[:, :, 1:, :]

In [55]:
# Augmenting Experimental Validated Data
augmented_input_expt_list, augmented_label_expt_list=augmenting_expt_val(input_expt_normalized,output_expt_normalized)
augmented_label_expt_list=list(np.array(augmented_label_expt_list).squeeze())

In [ ]:
# Appending Numerical and Val Expt Labels for Training

output_normalized=np.append(output_normalized,np.array(augmented_label_expt_list),axis=0)

In [ ]:
# Appending Numerical and Val Expt Data for Training

input_normalized=np.append(input_normalized,np.array(augmented_input_expt_list),axis=0)

In [56]:
#This is another augmenting by multiplying it by -1, itll seeda double the data set, dont do it again and again!!!!

# Appending Numerical and Val Expt Labels for Training

output_normalized=np.append(output_normalized,output_normalized,axis=0)

# Appending Numerical and Val Expt Data for Training

input_normalized=np.append(input_normalized,-1*input_normalized,axis=0)

## CNN

In [59]:
# print(input_normalized[50,:,:,:]+input_normalized[50+1013,:,:,:])
# print((output_normalized[50,:]-output_normalized[50+1013,:]))

In [60]:
## Input to CNN

print("Input normalized shape is:          {}".format(input_normalized.shape))
print("Output normalized shape is:         {}".format(output_normalized.shape))

Input normalized shape is:          (20350, 500, 4, 1)
Output normalized shape is:         (20350, 2)


In [61]:
## Experimental Validated to CNN

print("Input normalized shape is:          {}".format(input_expt_normalized.shape))
print("Output normalized shape is:         {}".format(output_expt_normalized.shape))

Input normalized shape is:          (24, 500, 4, 1)
Output normalized shape is:         (24, 2)


In [62]:
## Experimental Validated to CNN

print("Input normalized shape is:          {}".format(input_expt_test_normalized.shape))

Input normalized shape is:          (18, 500, 4, 1)


In [63]:
# #swapping 
# input_normalized[:,:,[0,1,2,3],:]=input_normalized[:,:,[0,2,1,3],:]
# input_expt_normalized[:,:,[0,1,2,3],:]=input_expt_normalized[:,:,[0,2,1,3],:]
# input_expt_test_normalized[:,:,[0,1,2,3],:]=input_expt_test_normalized[:,:,[0,2,1,3],:]

In [64]:
# output_normalized=output_normalized*75
# output_expt_normalized=output_expt_normalized*75

In [65]:
def CNN_arch(pool_filter_size, filter_size, stride, pool_filter_stride, 
             activation, kernel, input_normalized, learn_rate, regularizer, filters, initializer):
    
    """
    Function defining the architecture of the CNN model 
    """
    reg = tf.keras.regularizers.l1(regularizer)
    
    CNN_model = Sequential()
    
    
    CNN_model.add(Conv2D(int(1*filters), filter_size, strides = stride, activation = activation,
                         kernel_initializer = initializer, 
                         input_shape = (input_normalized.shape[1],input_normalized.shape[2],1), kernel_regularizer = reg))
    CNN_model.add(MaxPool2D(pool_size = pool_filter_size,strides = pool_filter_stride))
    
#     CNN_model.add(BatchNormalization())
    CNN_model.add(Conv2D(int(2*filters), filter_size, strides = stride, activation = activation,
                         kernel_initializer = initializer, kernel_regularizer = reg))
    CNN_model.add(MaxPool2D(pool_size = pool_filter_size,strides = pool_filter_stride))
    
#     CNN_model.add(BatchNormalization())
    CNN_model.add(Conv2D(2*filters, filter_size, strides = stride, activation = activation,
                         kernel_initializer = initializer, kernel_regularizer = reg))

    CNN_model.add(Conv2D(int(2*filters), (filter_size[0],2) , strides = (stride[0],2), activation = activation,
                         kernel_initializer = initializer, kernel_regularizer = reg))
    
    # start of the feedforward neural network 

    CNN_model.add(Flatten())
#     CNN_model.add(BatchNormalization())
    CNN_model.add(Dense(units = 2*filters, activation = activation, kernel_initializer = initializer, kernel_regularizer = reg))
    CNN_model.add(Dense(units = 16       , activation = activation, kernel_initializer = initializer, kernel_regularizer = reg))
    CNN_model.add(Dense(units = 2        , activation = activation, kernel_initializer = initializer, kernel_regularizer = reg))

    # optimizers for weights
    opt = Adam(learn_rate)

    CNN_model.compile(optimizer = opt, loss = 'mse', metrics = ['mse', 'mae','accuracy'])

    return CNN_model

In [66]:

def CNN_eval(epochs,pool_filter_size, filter_size, stride, pool_filter_stride,
             activation, kernel,input_normalized, output_normalized, learn_rate,
             regularizer, filters, initializer, split, batches,split_test):
    
    """
    Function for training the CNN 
    """

    # Creation of CNN model
    CNN_model = CNN_arch(pool_filter_size, filter_size, stride, pool_filter_stride, activation, kernel, 
                         input_normalized, learn_rate, regularizer, filters, initializer)

    # Splitting the test, train data set 

    x_train_1, x_test, y_train_1, y_test = train_test_split(input_normalized, output_normalized, test_size = split_test, 
                                                      shuffle = True)
    x_train, x_val, y_train, y_val = train_test_split(x_train_1, y_train_1, test_size = split, 
                                                      shuffle = True)


    print(CNN_model.summary())

    

    print('\n-------------------------------------------')
    print('               TEMP - Now final              ')
    print('---------------------------------------------')
    print("Subjects in Input Training:  {}".format(x_train.shape))
    print("Subjects in Output Training: {}".format(y_train.shape))
    print("Subjects in Input Testing:   {}".format(x_val.shape))
    print("Subjects in Output Testing:  {}".format(y_val.shape))
    print("Subjects in Input Testing:   {}".format(x_test.shape))
    print("Subjects in Output Testing:  {}".format(y_test.shape))

    # Fitting the CNN model
    start_time = time.time()
    CNN_history = CNN_model.fit(x_train, y_train,validation_data = (x_val,y_val), 
                                verbose = 1,batch_size = round(x_train.shape[0]/batches), epochs = epochs)
    
    print('Total run time: {}\n'.format(time.time()-start_time))
    
    print()
    print("Training Done!\n")
    
    # Evaluating Validation Data
    eval_1 = CNN_model.evaluate(x_val, y_val, verbose = 0)
    
    # Evaluating Training Data
    eval_2 = CNN_model.evaluate(x_train, y_train, verbose = 0)
    
    print('Validation Evaluation: Loss = {:.5f}, MSE = {:.5f}, MAE = {:.5f}, Accuracy = {:.5f}'. format(eval_1[0], eval_1[1], eval_1[2], eval_1[3]))
    print('Training Evaluation:   Loss = {:.5f}, MSE = {:.5f}, MAE = {:.5f}, Accuracy = {:.5f}\n'. format(eval_2[0], eval_2[1], eval_2[2], eval_2[3]))

    # Calculating the Mean square error for the training dataset
    mse_train = mean_squared_error(y_train, CNN_model.predict(x_train))

    # Calculating the Mean square error for the Validation dataset
    mse_val = mean_squared_error(y_val, CNN_model.predict(x_val))

    CNN_metrics_custom = np.array([[mse_train,mse_val]])

    print('Score: MSE Train = {:.5f}, MSE Validation = {:.5f}\n'.format(CNN_metrics_custom[0][0], CNN_metrics_custom[0][1]))

    return CNN_model, CNN_history, CNN_metrics_custom, x_test, y_test

In [71]:
# Defining the Hyper - parameters values 

learn_rate         = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate = 0.001,
                                                            decay_steps = 1000, decay_rate = 0.7)
num_epochs         = 30
filters            = 64 # Number of filters in First layer of CNN
regularizer        = 0.00000 # L1 Regularizer parameter value
filter_size        = (5,1) # Filter size of the CNN layers
stride             = (2,1) # 2D stride for the Filters
pool_filter_size   = (5,1) # Pooling filter size
pool_filter_stride = (2,1) # 2D pooling filter stride
activation         = 'tanh' 
initializer        = tf.keras.initializers.GlorotUniform() 
# initializer        = None # Weight initialization for the model
split              = 0.2 #Percentage of Train/Val Split
split_test         = 0.1 #Percentage of Test/Train Split
batches            = 1000
kernel             = ' '

In [ ]:
# Training the CNN model using local normalization 

M, H, C, x_test, y_test = CNN_eval(num_epochs, pool_filter_size, filter_size, stride, 
                         pool_filter_stride, activation, kernel, input_normalized, output_normalized,
                         learn_rate,regularizer,filters,initializer,split,batches,split_test)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 248, 4, 64)        384       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 122, 4, 64)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 59, 4, 128)        41088     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 28, 4, 128)       0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 12, 4, 128)        82048     
                                                                 
 conv2d_11 (Conv2D)          (None, 4, 2, 128)        

In [ ]:
# M = load_model(r".\Model_Own\Model_Goodone2.h")

In [ ]:
# Comparing the predicted and output label values (on Train + Val + Test Data)

print(np.max(np.abs((output_normalized - M.predict(input_normalized)) * 75)))

In [ ]:
# Comparing the predicted and output label values (on Test Data)

print(np.max(np.abs((y_test - M.predict(x_test)) * 75)))

In [ ]:
# Comparing the Experimental predicted and output label values  

np.set_printoptions(suppress=True)
print(np.max(np.abs(output_expt_normalized - M.predict(input_expt_normalized)) * 75))

In [ ]:
# Plotting the training and validation loss and accuracy parameters 

# fig, ax = plt.subplots(4, figsize = (30,45))

# # Plotting Training Parameters
# ax[0].plot(H.history['accuracy'], label = " Train Acc")
# ax[1].plot(H.history['mse'], label = "Train MSE", linewidth = 5)
# ax[2].plot(H.history['mae'], label = "Train MAE")
# ax[3].plot(H.history['loss'], label = "Train loss")

# # Plotting Validation Parameters
# ax[0].plot(H.history['val_accuracy'], label = "Val Acc")
# ax[1].plot(H.history['val_mse'], label = "Val MSE",linewidth = 5)
# ax[2].plot(H.history['val_mae'], label = "Val MAE")
# ax[3].plot(H.history['val_loss'], label = "Val loss")

# ax[1].set_xlabel('Epochs', fontsize = 20)
# ax[1].set_ylabel('MSE', fontsize = 20)
# ax[1].tick_params(axis='both', which='major', labelsize=20)
# ax[1].tick_params(axis='both', which='minor', labelsize=20)

# ax[0].grid()
# ax[1].grid()
# ax[2].grid()
# ax[3].grid()

# ax[0].legend(prop = {'size':25})
# ax[1].legend(prop = {'size':25})
# ax[2].legend(prop = {'size':25})
# ax[3].legend(prop = {'size':25})

# font_parameters = {'fontsize':25 , 'fontweight':'bold'}
# ax[0].set_title("Comparision of Accuracies", fontdict = font_parameters)
# ax[1].set_title("Comparision of Mean Square Errors", fontdict = font_parameters)
# ax[2].set_title("Comparision of Mean Absolute Errors", fontdict = font_parameters)
# ax[3].set_title("Comparision of Loss", fontdict = font_parameters)

In [ ]:
# RMS of Expt Validation Data
print("RMS of Expt Validation Data:")
print(mean_squared_error(output_expt_normalized*75, M.predict(input_expt_normalized)*75,squared=False))

In [ ]:
# RMS of CNN Test Data
print("RMS of CNN Test Data")
print(mean_squared_error(y_test*75, M.predict(x_test)*75,squared=False))

In [ ]:
# RMS of Whole Data
print("RMS of Whole Data")
print(mean_squared_error(output_normalized*75, M.predict(input_normalized)*75,squared=False))

In [ ]:
# R2 Score of Test Data
print("R2 Score of Test Data")
print(r2_score(y_test,M.predict(x_test)))

In [ ]:
# RMS of Whole Data

print("RMS of Whole Data")
print(mean_squared_error(output_expt_normalized*75, 5*np.round(M.predict(input_expt_normalized)*15),squared=False))
print(mean_squared_error(y_test*75, 5*np.round(M.predict(x_test)*15),squared=False))
print(mean_squared_error(output_normalized*75, 5*np.round(M.predict(input_normalized)*15),squared=False))

print(np.max(np.abs(output_expt_normalized*75-5*np.round(M.predict(input_expt_normalized)*15))))
print(np.max(np.abs(y_test*75- 5*np.round(M.predict(x_test)*15))))
print(np.max(np.abs(output_normalized*75- 5*np.round(M.predict(input_normalized)*15))))
# print((np.round(M.predict(input_normalized)*15)*5))
# print((M.predict(input_normalized)*75))
# print(((output_normalized)*75))

In [ ]:
# M.save(r".\Model_Own\Model_Goodone2_together_18.02.2022.h5")

# PREDICTIONS


In [ ]:
print("PREDICTIONS")

In [ ]:
print(np.max(np.abs(output_expt_test_given - (250 + 75 * M.predict(input_expt_test_normalized)))))

In [ ]:
print(mean_squared_error(output_expt_test_given-250, M.predict(input_expt_test_normalized)*75,squared=False))

In [ ]:
print(output_expt_test_given - (250 + 75 * M.predict(input_expt_test_normalized)))

In [ ]:
# Prediction
np.append(resampled_expt_test_label,M.predict(input_expt_test_normalized)*75+250,axis=1)

In [ ]:
print(mean_squared_error(output_expt_test_given-250, 5*np.round(M.predict(input_expt_test_normalized)*15),squared=False))
print(np.max(np.abs(output_expt_test_given-250- 5*np.round(M.predict(input_expt_test_normalized)*15))))

In [ ]:
# plot_saver_list_after(list(np.array(resampled_expt_test_data)[:,:,1:]),list(resampled_expt_test_label),'Expt_To_Check_Actual',200)

In [ ]:
# plot_saver_list_after(list(input_expt_normalized),list(output_expt_normalized*75),'Expt_Val_Plots',100)


In [ ]:
# plot_saver_list_after(list(normalized_inp[:, :, 1:, :]),list(normalized_out*75),'Num_Plots_only',50)

In [ ]:
# output_expt_normalized*75

In [ ]:
# plot_saver_list_after(list(input_expt_normalized[:4,   :,:,:]+normalized_inp[610, :, 1:, :]),list(output_expt_normalized[:4,   :]*75),'Differences',100)
# plot_saver_list_after(list(input_expt_normalized[4:8,  :,:,:]+normalized_inp[550, :, 1:, :]),list(output_expt_normalized[4:8,  :]*75),'Differences',100)
# plot_saver_list_after(list(input_expt_normalized[8:12, :,:,:]+normalized_inp[529, :, 1:, :]),list(output_expt_normalized[8:12, :]*75),'Differences',100)
# plot_saver_list_after(list(input_expt_normalized[12:16,:,:,:]+normalized_inp[280, :, 1:, :]),list(output_expt_normalized[12:16,:]*75),'Differences',100)
# plot_saver_list_after(list(input_expt_normalized[16:20,:,:,:]+normalized_inp[26, :,  1:, :]),list(output_expt_normalized[16:20,:]*75),'Differences',100)
# plot_saver_list_after(list(input_expt_normalized[20:24,:,:,:]+normalized_inp[837, :, 1:, :]),list(output_expt_normalized[20:24,:]*75),'Differences',100)

In [ ]:
# print(normalized_out[610, :]*75)
# print(normalized_out[550, :]*75)
# print(normalized_out[529, :]*75)
# print(normalized_out[280, :]*75)
# print(normalized_out[26, : ]*75)
# print(normalized_out[837, :]*75)

In [ ]:
# plot_saver_list_after(list(input_expt_normalized[:4,   :,:,:]+normalized_inp[610, :, 1:, :]),list(output_expt_normalized[:4,   :]*75),'Differences_2',100)
# plot_saver_list_after(list(input_expt_normalized[4:8,  :,:,:]+normalized_inp[550, :, 1:, :]),list(output_expt_normalized[4:8,  :]*75),'Differences_2',100)
# plot_saver_list_after(list(input_expt_normalized[8:12, :,:,:]+normalized_inp[529, :, 1:, :]),list(output_expt_normalized[8:12, :]*75),'Differences_2',100)
# plot_saver_list_after(list(input_expt_normalized[12:16,:,:,:]+normalized_inp[280, :, 1:, :]),list(output_expt_normalized[12:16,:]*75),'Differences_2',100)
# plot_saver_list_after(list(input_expt_normalized[16:20,:,:,:]+normalized_inp[26, :,  1:, :]),list(output_expt_normalized[16:20,:]*75),'Differences_2',100)
# plot_saver_list_after(list(input_expt_normalized[20:24,:,:,:]+normalized_inp[837, :, 1:, :]),list(output_expt_normalized[20:24,:]*75),'Differences_2',100)